This notebook is dedicated to scrapping Instagram data on the basis of a hashtag. It is freely based upon a tutorial [here](https://medium.com/@kseniatikhomirova/scrap-instagram-locations-with-python-d48ba6e56ebc). 

I have developed it with the hashtag "vanlife", since me and my wife have been interested in geography of this phenomenon.

In [1]:
import pandas as pd
import sys

### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import datetime as dt

from concurrent.futures import ThreadPoolExecutor

import sddk
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [ ]:
# further, to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
#PIA_data = gc.open_by_url()

# Hashtags

In [3]:
# count hashtag instances
hashtag = "vanlife"
url = "https://www.instagram.com/explore/tags/{0}/?__a=1".format(hashtag)
r = requests.get(url)
data = json.loads(r.text)
hashtag_count = data['graphql']['hashtag']['edge_hashtag_to_media']['count']
hastag_data_pages = hashtag_count / 75 # there is 75 instances per page
hashtag_count

7976500

In [10]:
data['graphql']['hashtag']['edge_hashtag_to_media']["edges"]

[{'node': {'comments_disabled': False,
   '__typename': 'GraphImage',
   'id': '2383435852566345967',
   'edge_media_to_caption': {'edges': [{'node': {'text': '💙GOLF R💙\n\nFOLLOW: @mr.carsxl \nOwner: @ebuzzq \n.\n.\n.\n.\n.\n.\n.\n.\n#mrcarsxl #like #volkswagon  #mk  #vw  #vwt  #golf  #volkfolk  #van  #dublife  #vwbus  #airlift  #gti  #cars  #golfr  #vwlove  #dirtydiesel  #t  #vanlife  #volkswagont  #r  #dubtillidie  #volkswagen  #car  #slammed  #kombi  #volkswagongolf  #vdub  #aircooled  #green'}}]},
   'shortcode': 'CETqkepJbjv',
   'edge_media_to_comment': {'count': 0},
   'taken_at_timestamp': 1598347716,
   'dimensions': {'height': 565, 'width': 1080},
   'display_url': 'https://scontent-frx5-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/118244291_1180403882345825_6001803233651554766_n.jpg?_nc_ht=scontent-frx5-1.cdninstagram.com&_nc_cat=105&_nc_ohc=cogsaV5rC9MAX8SUh_h&oh=0969c4dd73e24c2ca86abdea7da35dfc&oe=5F6D3647',
   'edge_liked_by': {'count': 0},
   'edge_media_preview_like'

In [ ]:
### maximal pages value
int(hastag_data_pages - 1)

67446

# Collecting end_cursors

In [17]:
def request_for_next_page(url): 
    r = requests.get(url)
    try:
        data = json.loads(r.text)
        end_cursor = data['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
    except:
        problem = "problem"
        print(problem)
        n = 0
        while (n <= 3 and problem == "problem"):  
            time.sleep(1)
            try: 
                r = requests.get(url)
                data = json.loads(r.text)
                end_cursor = data['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
                problem = "problem solved"
            except:
                n = n+1
    try:
        return end_cursor
    except:
        return "not-found"

In [22]:
try:
    # if we already have some endcursors data, we will start with them
    end_cursors = sddk.read_file("instagram_webscraping/end_cursors.json", "list", conf)
    print(end_cursors[-1])
except:
    # otherwise we will start with an empty list
    end_cursors = []
    print("starting with empty list")

file has not been found; check filename and path.
starting with empty list


In [ ]:
%%time
n_of_pages = 100
hashtag = "vanlife"
raw_url =  "https://www.instagram.com/explore/tags/" + hashtag + "/?__a=1"

end_cursor = ""

for n in range(n_of_pages):
    if len(end_cursors) > 0:
        url = raw_url + "&max_id=" + end_cursors[-1] # use the last end cursor
    else:
        url = raw_url
    actual_end_cursor = request_for_next_page(url)
    if actual_end_cursor !="not-found":
        end_cursors.append(actual_end_cursor)# value for the next page
        print(actual_end_cursor)
    else:
        break

QVFCT1dLZGtyQksyekRPUlJ3LVlaTFA0WFdUQ010OE9yNlJJU2p0cG8zUnNqa2pyemsyMU01M1h0dEdhNEUtN09hdG5JcFpBRTc3Y0FGVGEwMUR4b2Z4ag==
QVFCeWdGSzdzSmRMU25KenJJWXM1X3dUdmM3WXJOQ3hybG1SdE8zVVA4VnlQRXVFYThBQUZQVFVuUUF6dm5vWE0tcVY4UmNrdGRzeTNVeGFEUFRTT1VXcw==
QVFEbmRaeUdsNTZvR1BJX1BWYWdTM3JGRy1PMGNzSnk3dElmWmR6cy1HVG1TRGc3VFZXcU1VOE44ZGxUYk5SdFBuSzlUbWRBQTBwTGFrQ2E1U0lNQy1jVA==
QVFBNmdtbHhoak1YdUdlY2U5TDJRVnVQcUlEYWUzNWczY1U4VnNWUkxVR3poT1BXWXVuRF9TVmpRY3p5T3JBdklwLVFNdzdObHp3cS0tdTdiSmppa3pkSw==
QVFCVEFIVU4tWDluVjZuVXN4RHhjd1ZpREh1VWpJQ0ZhZ0kxMXJEazRJTldpVlphaUQ5bTY4a2ZsdHRfYzdWRWJCQ1h2N2ZIRXFjOWVnZmhySUJDUEJxRg==
QVFDd3FyZFM1NnlCcjMxMUNaaVhEb3J0S0JpU19fc29zTm1feVk0TXNxYm5QUnRuNGVEdEZQNEJ4U2hFQnJCNXFaaHkweHNrWW00NHB5T1lacnloc2tGYQ==
QVFCNXFmbUdyOHRxeUtXa1hZdEhaTVdNa2NRSHBHdlQzSl9WTXBvXzJXLWI1cWMzOHNaaFJYbm1zRTFrQ1pmcXlCSHdGeXZ5MVlnY0pPS0MzT2dnRnBobw==
QVFCa0ZyZjNVdkpVN3lJZXczT2xtTmxuRk9aU1BrTG5hdGRQYVNQTmhweWU4eHF1SlNicDBzOXBIdmtUZ09ESHU1NFFubEJpamVoNnlDSWNnNGF0czZIQw==
QVFBdjBpZmlGcExZcXMzSG9qc1A5TlNM

In [ ]:
sddk.write_file("instagram_webscraping/end_cursors.json", end_cursors, conf)

# Define crucial functions

In [ ]:
"https://www.instagram.com/p/BxKXnM5pptZ/?__a=1".partition("?__a=1")[0]

'https://www.instagram.com/p/BxKXnM5pptZ/'

In [ ]:
def mine_the_post(actual_url):
  post_data = json.loads(urlopen(actual_url).read().decode("utf-8"))
  try:
    location = post_data['graphql']['shortcode_media']['location']['slug']
    coordinates = gmaps.geocode(location)[0]["geometry"]["location"]
    lat = coordinates["lat"]
    lon = coordinates["lng"]
  except: 
    location = ""
    lat = ""
    lon = ""
  try: country_code = json.loads(post_data['graphql']['shortcode_media']['location']["address_json"])['country_code']
  except: country_code = ""
  try:
    timestamp = post_data['graphql']['shortcode_media']['taken_at_timestamp']
    timestamp = dt.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
  except: timestamp = ""  
  return [timestamp, country_code, location, lat, lon]

def deEmojify(inputString): # from here: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
  return inputString.encode('ascii', 'ignore').decode('ascii')

def get_post_info(item):
  item_node_shortcode = item["node"]['shortcode']
  post_url = "https://www.instagram.com/p/" + item_node_shortcode + "/?__a=1"
  try: 
    text = item['node']['edge_media_to_caption']['edges'][0]['node']['text'].replace("\n", " ")
    text = deEmojify(text)
  except: text = ""
  hashtags = []
  for word in text.split():
    if word.startswith("#"):
      hashtags.append(word.partition("#")[2])
  try: # produce a list of potential object on the picture
    caption = item['node']['accessibility_caption'].partition("contain: ")[2].split(", ")
    caption = caption[:-1] + caption[-1].split(" and ")
  except: caption = ""
  likes = item['node']['edge_liked_by']["count"]
  basic_data = [end_cursor, post_url.partition("?__a=1")[0], text, hashtags, caption, likes]
  try:
    url_data = mine_the_post(post_url)
  except:
    time.sleep(2)
    try: 
      url_data = mine_the_post(post_url)
    except:
      url_data = ["", "", "", "", ""]
  return basic_data + url_data

def get_edges(url_address):
  try: 
    r = requests.get(url_address)
    data = json.loads(r.text)
    edges = data['graphql']['hashtag']['edge_hashtag_to_media']['edges']
    return edges
  except: 
    try:
      time.sleep(3)
      r = requests.get(url_address)
      data = json.loads(r.text)
      edges = data['graphql']['hashtag']['edge_hashtag_to_media']['edges'] # list with posts
      return edges
    except:
      return "no edges"


In [ ]:
### to upload data into spreadsheet (not used anymore, but might be useful sometimes in the future)

last_sheet_position = 0
def update_cell_by_list(input_simple_list_data, last_position):
  number_of_added_rows = int(len(input_simple_list_data) / 11)
  new_last_row = last_position + number_of_added_rows
  new_range = "A" + str(int(last_position+1)) + ":K" + str(int(new_last_row))
  cell_list = worksheet.range(new_range)
  for cell, datapoint in zip(cell_list, input_simple_list_data):
    cell.value = str(datapoint)
  worksheet.update_cells(cell_list)
  #return new_last_row ### return me new value of the last row for next iteration 

# Start with preprocessed end_cursor data

In [ ]:
end_cursors_df = get_as_dataframe(sh.worksheet("end_cursors"))
end_cursors_df.head(2)

,0
0,QVFDcnRwZnlNaVM3YWpZRnJjZzZBNzM3ekU0Snh2eFlFeV...
1,QVFDUU5FN0tKMnN1U2hWaW5FaF9kbGZSd0RaZU04X3c3Qj...


In [ ]:
len(end_cursors_df)

1206

In [ ]:
end_cursors_list = end_cursors_df[0].tolist()
end_cursors_list[:3]

['QVFENnRnM0dqZ3AwdVRyNjlSZnQyUU4wV1ZKd1AxVEh0R0Y5MTBYRjlxVFc3OThSMml5a0tXQy10Ym5JS05IbDNUdERCQmRfQVlWOHQxNkxWd3R5QnFUUQ==',
 'QVFCYlF5TndQdE5jQkZOR0Qzb0RKY2w4UzJWZHhnd1BvQmVBVi1NOFhBc3hYa2piLU5sVDh0ek91OTdsWHgyTXVRdXdmUkJaYTI0Z01wWV94OGNSRVEzTQ==',
 'QVFCMG90MFFSNnVWMHBSLUhSTUpCZlBEYXhuLUd1dzFjOWwtUjlGT3hnZlRTX092RmlVY3BoUTh6NWszSWhzZ3VDb2VndDM0RXBOYVRTZnZYQV9HQUtfYg==']

In [ ]:
#sh.add_worksheet(title="vanlife_data", rows=300000, cols=11)
worksheet = sh.worksheet("vanlife_data")

In [ ]:
actual_data = []
tag = 'vanlife' # your tag
for end_cursor in end_cursors_list:
    url = "https://www.instagram.com/explore/tags/{0}/?__a=1&max_id={1}".format(hashtag, end_cursor)
    edges = get_edges(url)
    if edges != "no edges":
      with ThreadPoolExecutor(max_workers=75) as pool:
        current_parsed_edges = list(pool.map(get_post_info,edges))
      actual_data.extend(current_parsed_edges)
      print(end_cursor)
      if len(actual_data) >= 2000:
        actual_data_df = pd.DataFrame(actual_data)
        actual_data = []
        with open(gdrive_root +'vanlife_data.csv', 'a') as f:
          actual_data_df.to_csv(f, header=False)
### export our last data as well 
actual_data_df = pd.DataFrame(actual_data)
with open(gdrive_root +'vanlife_data.csv', 'a') as f:
  actual_data_df.to_csv(f, header=False)
        

# To begin with parsed data

In [ ]:
data_parsed_df = pd.read_csv(open(gdrive_root + 'vanlife_data.csv', 'r'), header=None)
data_parsed_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,QVFDcnRwZnlNaVM3YWpZRnJjZzZBNzM3ekU0Snh2eFlFeV...,BxKXnM5pptZ,I spent the majority of the day under the bus....,"['dirty', 'greasemonkey', 'greasy', 'skoolie',...","['1 person', 'beard', 'outdoor']",77,2019-05-07 12:53:18,NaN,NaN,NaN,NaN
1,1,QVFDcnRwZnlNaVM3YWpZRnJjZzZBNzM3ekU0Snh2eFlFeV...,BxKXj9kl-40,So this a bit of a shock was expecting them to...,"['ryobitools', 'vanproject', 'vanlifecommunity...","['people sitting', 'indoor']",46,2019-05-07 12:52:51,GB,woodbridge-suffolk,52.095481,1.312571
2,2,QVFDcnRwZnlNaVM3YWpZRnJjZzZBNzM3ekU0Snh2eFlFeV...,BxKXivglfDT,| This year and last | 2019 & 2018 - Poppy cou...,"['t3camper', 'vanlife', 'buslife', 'truckfest'...","['sky', 'outdoor']",14,2019-05-07 12:52:41,NaN,NaN,NaN,NaN


In [ ]:
# check the size of the dataframe
def get_size(input):
  return print(str(sys.getsizeof(input) * 0.000001) + " MB")
get_size(data_parsed_df)

103.930213 MB


In [ ]:
len(data_parsed_df)

79637

In [ ]:
### to check how the data look like
for number in range(0, 79637, 1000):
  print(data_parsed_df.iloc[number][7])

# Backup


In [ ]:
actual_data = []
tag = 'vanlife' # your tag
for end_cursor in end_cursors_list[:1]:
    url = "https://www.instagram.com/explore/tags/{0}/?__a=1&max_id={1}".format(hashtag, end_cursor)
    r = requests.get(url)
    data = json.loads(r.text)
    edges = data['graphql']['hashtag']['edge_hashtag_to_media']['edges'] # list with posts
    for item in edges:
      item_node_shortcode = item["node"]['shortcode']
      try: text = item['node']['edge_media_to_caption']['edges'][0]['node']['text'].replace("\n", " ")
      except: text = ""
      hashtags = []
      for word in text.split():
        if word.startswith("#"):
          hashtags.append(word.partition("#")[2])
      try: # produce a list of potential object on the picture
        caption = item['node']['accessibility_caption'].partition("contain: ")[2].split(", ")
        caption = caption[:-1] + caption[-1].split(" and ")
      except: caption = ""
      likes = item['node']['edge_liked_by']["count"]
      post_url = "https://www.instagram.com/p/" + item_node_shortcode + "/?__a=1"
      try:
        post_data = mine_the_post(post_url)
        actual_data.append(post_data)
      except:
        pass

In [ ]:
actual_data

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
actual_data = []
tag = 'vanlife' # your tag
page_count = 1000  # int(hastag_data_pages - 1)
for url in range(0, page_count):
    url = "https://www.instagram.com/explore/tags/{0}/?__a=1&max_id={1}".format(hashtag, end_cursor)
    r = requests.get(url)
    data = json.loads(r.text)
    end_cursor = data['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor'] # value for the next page
    edges = data['graphql']['hashtag']['edge_hashtag_to_media']['edges'] # list with posts
    with ThreadPoolExecutor(max_workers=50) as pool:
      current_parsed_edges = list(pool.map(get_post_info,edges))
    actual_data.extend(current_parsed_edges)
    print(end_cursor)
    if len(actual_data) >= 1000:
      actual_data_merged = []
      for element in actual_data:
        actual_data_merged.extend(element)
      update_cell_by_list(actual_data_merged, last_sheet_position)
      number_of_added_rows = len(actual_data_merged) / 11
      new_last_row = last_sheet_position + number_of_added_rows
      last_sheet_position = new_last_row
      actual_data = []
      actual_data_merged = []
      
        


QVFEakJ5UEtjM2tsdGNPU2Nqa3l5aWh1aVlaTmlJY3YxUEtUWWFVUEw4aFdJMFQySGhUcjlXS1pncVkwazNvSHNfQkZPVlI2akozb3NkeWExYVN4M1g5YQ==


KeyboardInterrupt: ignored

In [ ]:
last_sheet_position = 0
def update_cell_by_list(input_simple_list_data, last_position):
  number_of_added_rows = int(len(input_simple_list_data) / 8)
  new_last_row = last_position + number_of_added_rows
  new_range = "A" + str(int(last_position+1)) + ":H" + str(int(new_last_row))
  cell_list = worksheet.range(new_range)
  for cell, datapoint in zip(cell_list, input_simple_list_data):
    cell.value = str(datapoint)
  worksheet.update_cells(cell_list)
  #return new_last_row ### return me new value of the last row for next iteration 

In [ ]:
last_sheet_position = 0
for testing_list in [data_testing_merged, data_testing_merged, data_testing_merged]:
  update_cell_by_list(testing_list, last_sheet_position)
  number_of_added_rows = len(testing_list) / 8
  new_last_row = last_sheet_position + number_of_added_rows
  last_sheet_position = new_last_row

In [ ]:
# Select a range
cell_list = worksheet.range('A101:H200')

for cell, datapoint in zip(cell_list, data_testing_merged):
    cell.value = str(datapoint)

# Update in batch
worksheet.update_cells(cell_list)

{'spreadsheetId': '1OZySlFGa67xcIyZppptnaNuxFqNwx6AfkPUTHOUcjJc',
 'updatedCells': 800,
 'updatedColumns': 8,
 'updatedRange': 'testing_MAY7!A101:H200',
 'updatedRows': 100}

In [ ]:
sh.values_update(
    "SheetA1:H100"+str(len(data_testing)), 
    params={'valueInputOption': 'RAW'}, 
    body={"range": "A1:B"+str(len(arr_test)), "majorDimension": "ROWS", "values": arr_test,
}
)

NameError: ignored

In [ ]:
url = https://sheets.googleapis.com/v4/spreadsheets/{spreadsheetId}/values/{range}:append

    
body = {
  "range": range,
  "majorDimension": "ROWS",
  "values": arr_test,
}

In [ ]:
arr_test[:2]